In [2]:
from InstanceClass import *

In [8]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [5]:
instance = Instance(path="toy_instance.json")
instance.build_instance()

In [7]:
instance.variables

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

creation of 𝐴 = {𝑎_{𝑖,𝑗,𝑘,l}} : la matrice des affections des collaborateurs de taille 𝑛𝑐 ∗ ℎ  ∗ 𝑛𝑝 ∗ 𝑛

On en extrait une matrice 𝐴𝑝 = {𝑎𝑝𝑗,𝑘} indiquant si à l’instant j le projet vient d’être
rendu 


In [78]:
I=3
J=5
L=5
K=3

In [93]:
m = gp.Model("matrix1")

# Create variables
A = m.addMVar(shape=(3, 5, 5, 3), vtype=GRB.BINARY, name="A")

In [83]:
A

{(0, 0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 4, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1, 0): <gurobi.Var *Awaiting Model Upd

In [350]:
def solution_creation(variables):
    shape=(variables["NC"],variables["H"],variables["NP"],variables["NA"])
    return shape

def calcul_work(Affectation):
    rep = [np.sum([Affectation[i][j] for i in range(3)],axis=0) for j in range(np.shape(Affectation)[1])]
    return np.array(rep)

def calcul_temporel_work(work,horizon):
    rep = [[gp.quicksum(work_project[0:(i+1)]) for i in range(horizon)] for work_project in work]
    return np.array(rep)

def calcul_work_final(work):
    rep=gp.quicksum(work)  
    return rep

def calcul_Ap(rep,cost_projet):
    X = [[(rep[i][j] >= cost_projet[i]) for j in range(len(rep))] for i in range(len(cost_projet))]
    return np.array(X)

In [ ]:
model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((Affectation.sum(i,j, '*') <= 1 for i in range(I) for j in range(J)), name="Quantity_worked_Constraint") 
model.addConstrs((Affectation.sum(i,j, '*') <= 1-CONGES[i][j] for i in range(I) for j in range(J)), name="Day_off_Constraint") 


In [353]:
calcul_temporel_work(calcul_work(A),5)[0][4][0]

<MLinExpr ()>
array( <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>)

In [330]:
calcul_work(A)[0][0]+calcul_work(A)[0][1]

<MLinExpr (3,)>
array([ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
        <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
        <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>])

In [339]:
np.shape(calcul_work(A))

(5,)

In [340]:
calcul_work(A)

array([<MLinExpr (5, 3)>
       array([[ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
                <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
                <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>],
              [ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
                <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
                <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>],
              [ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
                <gurobi.Var *Awaiting Model Updat

In [358]:
instance.variables

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

In [361]:
shape

(3, 5, 5, 3)

In [374]:
(I,J,K,L)

(3, 5, 3, 5)

In [375]:
shape = solution_creation(instance.variables)
COST_PROJECT = instance.variables["COST_PROJECT"]
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]


model = gp.Model("matrix1")

# Create variables
Affectation = model.addMVar(shape=(I,J,L,K), vtype=GRB.BINARY, name="Affectation")
Done_Project = model.addMVar(shape=L, vtype=GRB.BINARY, name="Done_Project")
Ap = calcul_Ap(calcul_work(Affectation),COST_PROJECT)
final_work = calcul_work_final(calcul_work(Affectation))
temp_work = calcul_temporel_work(calcul_work(Affectation),instance.variables["H"])

model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((gp.quicksum(Affectation[i][j]) <= 1 for i in range(I) for j in range(J)), name="Quantity_worked_Constraint") 
model.addConstrs((gp.quicksum(Affectation[i][j]) <= 1-CONGES[i][j] for i in range(I) for j in range(J)), name="Day_off_Constraint") 

model.addConstrs((Done_Project[i] <= 1 for i in range(L)), name="Only_Done_One_Time") 


{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>}

In [387]:
model.addGenConstrIndicator(Done_Project[i], 1, final_work[i], GRB.S, 1, name="Only_Done_One_Time") 


ValueError: can only convert an array of size 1 to a Python scalar

In [383]:
model.addGenConstrIndicator((Done_Project[i], 1, final_work[i], GRB.EQUAL, 1 for i in range(L)), name="Implication_Constraint")


SyntaxError: invalid syntax (<ipython-input-383-1410d1cee77f>, line 1)

In [377]:
final_work

<MLinExpr (5, 3)>
array([[ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model U

In [376]:
Done_Project

<MVar (5,)>
array([<gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>])

In [363]:
model.addConstrs((Affectation.sum(i,j, '*') <= 1 for i in range(I) for j in range(J)), name="Quantity_worked_Constraint")


TypeError: sum() takes at most 2 positional arguments (4 given)

In [357]:
np.shape(temp_work)

(5, 5)

In [314]:
work = calcul_work(A)
gp.quicksum(work)

<MLinExpr (5, 3)>
array([[ <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>,
         <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model U

In [303]:
[[1 if (work[i][j] - COST_PROJECT[i])>=0 else 0 for j in range(len(work))] for i in range(len(COST_PROJECT))]


GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

In [283]:
shape

(3, 5, 5, 3)

Constraint 1

In [ ]:
model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((Affectation.sum(i,j, '*') <= 1 for i in range(I) for j in range(J)), name="Quantity_worked_Constraint") 
model.addConstrs((Affectation.sum(i,j, '*') <= 1-CONGES[i][j] for i in range(I) for j in range(J)), name="Day_off_Constraint") 


In [ ]:
model.addConstrs((Affectation.sum(i,j, '*') <= 1-CONGES[i][j] for i in range(I) for j in range(J)), name="Day_off_Constraint") 


Constraint 2

In [ ]:
model.addConstrs((A.sum(i,j, '*') <= 1 for i in range(I) for j in range(J)), name="Quantity_worked_Constraint") 

In [47]:
STAFF_QUALIFICATIONS[0]

array([1., 1., 1.])

In [57]:
Affectation[0][0]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
H

In [56]:
Affectation[0][0][0]

array([0., 0., 0.])